In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
date = session.query(Measurement, Measurement.date).order_by(Measurement.date.desc()).limit(1)
date[0][1]

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
most_recent_date = date[0][1]

# Calculate the date one year from the last date in data set.
one_year_date = dt.date(2017, 8 ,23) - dt.timedelta(days = 365)

# Perform a query to retrieve the data and precipitation scores
query = session.query(Measurement.date, Measurement.prcp).\
            filter(Measurement.date >= one_year_date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(query)
df = df.rename({0:"date", 1:"prcp"}, axis = 1)
df.set_index("date", inplace = True)

# Sort the dataframe by date
df = df.sort_index() 

# Use Pandas Plotting with Matplotlib to plot the data
fig,axes = plt.subplots(1,1)
fig.set_size_inches(12, 4)
axes.bar(df.index, df['prcp'])
plt.xlabel("Date")
plt.ylabel("Precipitation (Inches)")
plt.xticks(rotation = 90)
axes.xaxis.set_major_locator(MaxNLocator(10)) 
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count=session.query(func.count(Station.id)).all()
station_count

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
session.query(Measurement.station, func.count(Measurement.id)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.id).desc()).all()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = session.query(Measurement.station, func.count(Measurement.id)).\
                        group_by(Measurement.station).\
                        order_by(func.count(Measurement.id).desc()).limit(1)[0][0]
session.query(Measurement.station).\
    filter(Measurement.station == most_active_station).\
    with_entities(Measurement.station, func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temps = session.query(Measurement.tobs).\
            filter(Measurement.station == most_active_station).\
            filter(Measurement.date >= one_year_date).all()
df = pd.DataFrame(temps)
fig,axes = plt.subplots(1,1)
fig.set_size_inches(8, 4)
axes.hist(df[0], bins = 12)
plt.xlabel("Temperature")
plt.ylabel("Frequency")
axes.xaxis.set_major_locator(MaxNLocator(6)) 
plt.tight_layout()
plt.show()


# Close session

In [ ]:
# Close Session
session.close()